#### Importing Packages

In [ ]:
import whisper 
import pandas as pd 
import torch 
from transformers import pipeline 
from huggingface_hub import model_info 
from tqdm.notebook import tqdm
import time
import pandas as pd
import pyarabic.araby as araby 
import pyarabic.number as number
import re
from pathlib import Path
import locale 
locale.getpreferredencoding = lambda: 'UTF-8' 
import jiwer

##### Text Cleaning Function

In [ ]:
def remove_files(text):
    result = []
    regex = re.compile("\[.*?\]")
    temp_result = re.findall(regex, text)
    en_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for string in temp_result:
        contains = any((c in string) for c in en_letters)
        if contains:
            result.append(string)
    for el in result:
        text = text.replace(el,' ')
    return text
def clean_some_chars(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!'
              ,"[","]","{","}","*",":","#","$","€","£","~","<",">","/","|","'",",",'=','(',')','+','•',';','&','–','♦','%'
             ,'»','»','·']
    
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ',' ',' '
               ," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ",' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
              ,' ',' ',' ']
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace('\'','')
    # removing numbers
    text = ''.join([i for i in text if not i.isdigit()])
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text
def clean_english_chars(text):
    search = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for i in range(0, len(search)):
        text = text.replace(search[i],' ')
    return text
def remove_unnecessary_spaces(text):
    return re.sub(' +',' ',text)
def remove_non_arabic_letters(text):
    
    '''
    ALEF_MADDA       = u'\u0622' 
    ALEF_HAMZA_ABOVE = u'\u0623' 
    WAW_HAMZA        = u'\u0624' 
    ALEF_HAMZA_BELOW = u'\u0625' 
    YEH_HAMZA        = u'\u0626' 
    ALEF             = u'\u0627' 
    BEH              = u'\u0628' 
    TEH_MARBUTA      = u'\u0629' 
    TEH              = u'\u062a' 
    THEH             = u'\u062b' 
    JEEM             = u'\u062c' 
    HAH              = u'\u062d' 
    KHAH             = u'\u062e' 
    DAL              = u'\u062f' 
    THAL             = u'\u0630' 
    REH              = u'\u0631' 
    ZAIN             = u'\u0632' 
    SEEN             = u'\u0633' 
    SHEEN            = u'\u0634' 
    SAD              = u'\u0635' 
    DAD              = u'\u0636' 
    TAH              = u'\u0637' 
    ZAH              = u'\u0638' 
    AIN              = u'\u0639' 
    GHAIN            = u'\u063a' 
    TATWEEL          = u'\u0640' 
    FEH              = u'\u0641' 
    QAF              = u'\u0642' 
    KAF              = u'\u0643' 
    LAM              = u'\u0644' 
    MEEM             = u'\u0645' 
    NOON             = u'\u0646' 
    HEH              = u'\u0647' 
    WAW              = u'\u0648' 
    ALEF_MAKSURA     = u'\u0649' 
    YEH              = u'\u064a' 
    MADDA_ABOVE      = u'\u0653' 
    HAMZA_ABOVE      = u'\u0654' 
    HAMZA_BELOW      = u'\u0655' 
    LAM_ALEF                     = u'\ufefb' 
    LAM_ALEF_HAMZA_ABOVE         = u'\ufef7' 
    LAM_ALEF_HAMZA_BELOW         = u'\ufef9' 
    LAM_ALEF_MADDA_ABOVE         = u'\ufef5' 
    '''
    
    regex = re.compile(r'[\u0622\u0623\u0624\u0625\u0626\u0627\u0628\u0629\u062a\u062b\u062c\u062d\u062e\u062f\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063a\u0640\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u0649\u064a\u0653\u0654\u0655\ufefb\ufef7\ufef9\ufef5]')
    # removing Arabic letters from the text and storing the result in the varialbe: unwanted_str .
    unwanted_str = regex.sub(' ',text)
    # Creating a list containing all of the unwanted characters, letters and symbols.
    unwanted_list_of_strs = list(unwanted_str.replace(" ", ""))
    # Cleaning the unwanted list of characters out of the text
    for i in range(0, len(unwanted_list_of_strs)):
        text = text.replace(unwanted_list_of_strs[i], " ")
    
    text = remove_unnecessary_spaces(text)
    
    return text
def concatenate_list_into_string(lis_strs):
    result = ""
    for el in lis_strs:
        result += " " + el
    return result
def remove_single_letters(text):
    words = text.split(' ')
    waw = 'و'
    for word in words:
        if len(word.strip()) == 1:
            if word != waw:
                words.remove(word)
    text = concatenate_list_into_string(words)
    return text
def clean_text(text):
    # removing files
    text = remove_files(text)
    # removing some unuseful chars
    text = clean_some_chars(text)
    # removing english chars
    text = clean_english_chars(text)
    # removing tashkeel
    text = araby.strip_tashkeel(text)
    # removing longation
    text = araby.strip_tatweel(text)
    # removing unwanted spaces
    text = remove_unnecessary_spaces(text)
    # removing non-arabic characters
    text = remove_non_arabic_letters(text)
    # removing single unwanted letters
    text = remove_single_letters(text)
    # returning result
    return text    

#### Loading Whisper and WhisperAR models 

In [ ]:
#Quran Reference, retrieved from tanzil.net 
df_ref = pd.read_csv("quran-simple-clean.txt",header=None) 
vals_groundtruth = df_ref[0].values

In [ ]:
#OpenAI's Whisper - Large ~ requires 10GB of VRAM, otherwise it will through a CUDA out of memory error
model = whisper.load_model("large") 

In [ ]:
Transcriptions_list = [] 
file_list = [] 
time_list = [] 
directory = '/Path to .wav audio files' 
files = sorted(Path(directory).glob('*.wav')) 
for i, file in tqdm(enumerate(files)): 
    start = time.time() 
    Transcriptions_list.append(model.transcribe(str(file),language='ar')['text']) 
    end = time.time() 
    file_list.append(str(file).split('/')[-1]) 
    time_list.append(end-start) 
df = pd.DataFrame({'File_no':file_list,
                   'Transcription':Transcriptions_list,
                   'Execution_time':time_list})
df['Transcription'] = df['Transcription'].apply(clean_text) 
vals = df['Transcription'].values

In [ ]:
#Evaluate against Quran reference 
wer_list = [] 
mer_list = [] 
cer_list = [] 
wil_list = [] 
for i, j in zip(vals,vals_groundtruth): 
  wer_list.append(jiwer.wer(j,i)) 
  mer_list.append(jiwer.mer(j,i)) 
  cer_list.append(jiwer.cer(j,i)) 
  wil_list.append(jiwer.wil(j,i))
df['WER'] = wer_list 
df['MER'] = mer_list 
df['CER'] = cer_list 
df['WIL'] = wil_list 
df.to_csv("whisper_eval.csv",index=False) 

In [ ]:
#WhisperAR - Large 
MODEL_NAME = 'arbml/whisper-largev2-ar'
lang = 'ar'
device = 0 if torch.cuda.is_available() else 'cpu' 
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

In [ ]:
transcription_list = [] 
file_list = [] 
time_list = [] 
directory = '/Path to .wav audio files'
files = sorted(Path(directory).glob('*.wav'))
for i, file in tqdm(enumerate(files)): 
  start = time.time() 
  transcription_list.append(pipe(str(file))['text'])
  end = time.time() 
  file_list.append(str(file).split('/')[-1])
  time_list.append(end-start)
import pandas as pd 
df = pd.DataFrame({'File_no':file_list, 
                   'Transcription':transcription_list, 
                   'Execution_time':time_list}) 
df['Transcription'] = df['Transcription'].apply(clean_text) 
vals = df['Transcription'].values

In [ ]:
#Evaluate against Quran reference 
wer_list = [] 
mer_list = [] 
cer_list = [] 
wil_list = [] 
for i, j in zip(vals,vals_groundtruth): 
  wer_list.append(jiwer.wer(j,i)) 
  mer_list.append(jiwer.mer(j,i)) 
  cer_list.append(jiwer.cer(j,i)) 
  wil_list.append(jiwer.wil(j,i))
df['WER'] = wer_list 
df['MER'] = mer_list 
df['CER'] = cer_list 
df['WIL'] = wil_list 
df.to_csv("whisperAR_eval.csv",index=False) 

In [ ]:
#Tarteel - base
MODEL_NAME = 'tarteel-ai/whisper-base-ar-quran'
lang = 'ar'
device = 0 if torch.cuda.is_available() else 'cpu' 
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

In [ ]:
transcription_list = [] 
file_list = [] 
time_list = [] 
directory = '/Path to .wav audio files'
files = sorted(Path(directory).glob('*.wav'))
for i, file in tqdm(enumerate(files)): 
  start = time.time() 
  transcription_list.append(pipe(str(file))['text'])
  end = time.time() 
  file_list.append(str(file).split('/')[-1])
  time_list.append(end-start)
import pandas as pd 
df = pd.DataFrame({'File_no':file_list, 
                   'Transcription':transcription_list, 
                   'Execution_time':time_list}) 
df['Transcription'] = df['Transcription'].apply(clean_text) 
vals = df['Transcription'].values

In [ ]:
#Evaluate against Quran reference 
wer_list = [] 
mer_list = [] 
cer_list = [] 
wil_list = [] 
for i, j in zip(vals,vals_groundtruth): 
  wer_list.append(jiwer.wer(j,i)) 
  mer_list.append(jiwer.mer(j,i)) 
  cer_list.append(jiwer.cer(j,i)) 
  wil_list.append(jiwer.wil(j,i))
df['WER'] = wer_list 
df['MER'] = mer_list 
df['CER'] = cer_list 
df['WIL'] = wil_list 
df.to_csv("tarteel_base_eval.csv",index=False) 

In [ ]:
#Tarteel - tiny
MODEL_NAME = 'tarteel-ai/whisper-tiny-ar-quran'
lang = 'ar'
device = 0 if torch.cuda.is_available() else 'cpu' 
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

In [ ]:
transcription_list = [] 
file_list = [] 
time_list = [] 
directory = '/Path to .wav audio files'
files = sorted(Path(directory).glob('*.wav'))
for i, file in tqdm(enumerate(files)): 
  start = time.time() 
  transcription_list.append(pipe(str(file))['text'])
  end = time.time() 
  file_list.append(str(file).split('/')[-1])
  time_list.append(end-start)
import pandas as pd 
df = pd.DataFrame({'File_no':file_list, 
                   'Transcription':transcription_list, 
                   'Execution_time':time_list}) 
df['Transcription'] = df['Transcription'].apply(clean_text) 
vals = df['Transcription'].values

In [ ]:
#Evaluate against Quran reference 
wer_list = [] 
mer_list = [] 
cer_list = [] 
wil_list = [] 
for i, j in zip(vals,vals_groundtruth): 
  wer_list.append(jiwer.wer(j,i)) 
  mer_list.append(jiwer.mer(j,i)) 
  cer_list.append(jiwer.cer(j,i)) 
  wil_list.append(jiwer.wil(j,i))
df['WER'] = wer_list 
df['MER'] = mer_list 
df['CER'] = cer_list 
df['WIL'] = wil_list 
df.to_csv("tarteel_tiny_eval.csv",index=False) 